In [1]:
import sys
import os
import pretrainedmodels

In [2]:
#!pip install albumentations==0.0.10

In [3]:
import torch
import albumentations
from PIL import Image
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.nn import functional as F

In [4]:
TEST_BATCH_SIZE = 1
MODEL_MEAN = (0.485, 0.456, 0.405)
MODEL_STD = (0.229, 0.224, 0.225)
IMG_HEIGHT = 512
IMG_WIDTH = 512
DEVICE = "cuda"

In [5]:
class ResNet34(nn.Module):
    def __init__(self, pretrained):
        super(ResNet34, self).__init__()
        if pretrained is True:
            self.model = pretrainedmodels.__dict__["resnet34"](pretrained="imagenet")
        else:
            self.model = pretrainedmodels.__dict__["resnet34"](pretrained=None)
        
        self.l0 = nn.Linear(512, 2)  # nucleoplasmn
        self.l1 = nn.Linear(512, 2)  # nuclear_membrane
        self.l2 = nn.Linear(512, 2)  # nucleoli
        self.l3 = nn.Linear(512, 2)  # nucleoli_fibrillar_center
        self.l4 = nn.Linear(512, 2)  # nuclear_speckles
        self.l5 = nn.Linear(512, 2)  # nuclear_bodies
        self.l6 = nn.Linear(512, 2)  # endoplasmic_reticulum
        self.l7 = nn.Linear(512, 2)  # golgi_apparatus
        self.l8 = nn.Linear(512, 2)  # peroxisomes
        self.l9 = nn.Linear(512, 2)  # endosomes
        self.l10 = nn.Linear(512, 2)  # lysosomes
        self.l11 = nn.Linear(512, 2)  # intermediate_filaments
        self.l12 = nn.Linear(512, 2)  # actin_filaments
        self.l13 = nn.Linear(512, 2)  # focal_adhesion_sites
        self.l14 = nn.Linear(512, 2)  # microtubules
        self.l15 = nn.Linear(512, 2)  # microtubule_ends
        self.l16 = nn.Linear(512, 2)  # cytokinetic_bridge
        self.l17 = nn.Linear(512, 2)  # mitotic_spindle
        self.l18 = nn.Linear(512, 2)  # microtubule_organizing_center
        self.l19 = nn.Linear(512, 2)  # centrosome
        self.l20 = nn.Linear(512, 2)  # lipid_droplets
        self.l21 = nn.Linear(512, 2)  # plasma_membrane
        self.l22 = nn.Linear(512, 2)  # cell_junctions
        self.l23 = nn.Linear(512, 2)  # mitochondria
        self.l24 = nn.Linear(512, 2)  # aggresome
        self.l25 = nn.Linear(512, 2)  # cytosol
        self.l26 = nn.Linear(512, 2)  # cytoplasmic_bodies
        self.l27 = nn.Linear(512, 2)  # rods_rings
        
    def forward(self, x):
        #print(x.shape)
        bs, _, _, _ = x.shape
        x = self.model.features(x)
        # Adaptive pooling supports all image sizes
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        l0  = self.l0(x)
        l1  = self.l1(x)
        l2  = self.l2(x)
        l3  = self.l3(x)
        l4  = self.l4(x)
        l5  = self.l5(x)
        l6  = self.l6(x)
        l7  = self.l7(x)
        l8  = self.l8(x)
        l9  = self.l9(x)
        l10  = self.l10(x)
        l11  = self.l11(x)
        l12  = self.l12(x)
        l13  = self.l13(x)
        l14  = self.l14(x)
        l15  = self.l15(x)
        l16  = self.l16(x)
        l17  = self.l17(x)
        l18  = self.l18(x)
        l19  = self.l19(x)
        l20  = self.l20(x)
        l21  = self.l21(x)
        l22  = self.l22(x)
        l23  = self.l23(x)
        l24  = self.l24(x)
        l25  = self.l25(x)
        l26  = self.l26(x)
        l27  = self.l27(x)

        return l0, l1, l2, l3, l4, l5, l6, l7, l8, l9, l10, l11, l12, l13, l14, l15, l16, l17, l18, l19, l20, l21, l22, l23, l24, l25, l26, l27

In [6]:
class ResNet50(nn.Module):
    def __init__(self, pretrained):
        super(ResNet50, self).__init__()
        if pretrained is True:
            self.model = pretrainedmodels.__dict__["resnet50"](pretrained="imagenet")
        else:
            self.model = pretrainedmodels.__dict__["resnet50"](pretrained=None)
        
        self.l0 = nn.Linear(2048, 2)  # nucleoplasmn
        self.l1 = nn.Linear(2048, 2)  # nuclear_membrane
        self.l2 = nn.Linear(2048, 2)  # nucleoli
        self.l3 = nn.Linear(2048, 2)  # nucleoli_fibrillar_center
        self.l4 = nn.Linear(2048, 2)  # nuclear_speckles
        self.l5 = nn.Linear(2048, 2)  # nuclear_bodies
        self.l6 = nn.Linear(2048, 2)  # endoplasmic_reticulum
        self.l7 = nn.Linear(2048, 2)  # golgi_apparatus
        self.l8 = nn.Linear(2048, 2)  # peroxisomes
        self.l9 = nn.Linear(2048, 2)  # endosomes
        self.l10 = nn.Linear(2048, 2)  # lysosomes
        self.l11 = nn.Linear(2048, 2)  # intermediate_filaments
        self.l12 = nn.Linear(2048, 2)  # actin_filaments
        self.l13 = nn.Linear(2048, 2)  # focal_adhesion_sites
        self.l14 = nn.Linear(2048, 2)  # microtubules
        self.l15 = nn.Linear(2048, 2)  # microtubule_ends
        self.l16 = nn.Linear(2048, 2)  # cytokinetic_bridge
        self.l17 = nn.Linear(2048, 2)  # mitotic_spindle
        self.l18 = nn.Linear(2048, 2)  # microtubule_organizing_center
        self.l19 = nn.Linear(2048, 2)  # centrosome
        self.l20 = nn.Linear(2048, 2)  # lipid_droplets
        self.l21 = nn.Linear(2048, 2)  # plasma_membrane
        self.l22 = nn.Linear(2048, 2)  # cell_junctions
        self.l23 = nn.Linear(2048, 2)  # mitochondria
        self.l24 = nn.Linear(2048, 2)  # aggresome
        self.l25 = nn.Linear(2048, 2)  # cytosol
        self.l26 = nn.Linear(2048, 2)  # cytoplasmic_bodies
        self.l27 = nn.Linear(2048, 2)  # rods_rings
        
    def forward(self, x):
        bs, _, _, _ = x.shape
        x = self.model.features(x)
        # print(x.shape) # The dim is: bs * 2048 * 16 * 16 for images of size 3*512*512
        # Adaptive pooling supports all image sizes
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        # print(x.shape) # The dim is: bs * 2048
        l0  = self.l0(x)
        l1  = self.l1(x)
        l2  = self.l2(x)
        l3  = self.l3(x)
        l4  = self.l4(x)
        l5  = self.l5(x)
        l6  = self.l6(x)
        l7  = self.l7(x)
        l8  = self.l8(x)
        l9  = self.l9(x)
        l10  = self.l10(x)
        l11  = self.l11(x)
        l12  = self.l12(x)
        l13  = self.l13(x)
        l14  = self.l14(x)
        l15  = self.l15(x)
        l16  = self.l16(x)
        l17  = self.l17(x)
        l18  = self.l18(x)
        l19  = self.l19(x)
        l20  = self.l20(x)
        l21  = self.l21(x)
        l22  = self.l22(x)
        l23  = self.l23(x)
        l24  = self.l24(x)
        l25  = self.l25(x)
        l26  = self.l26(x)
        l27  = self.l27(x)

        return l0, l1, l2, l3, l4, l5, l6, l7, l8, l9, l10, l11, l12, l13, l14, l15, l16, l17, l18, l19, l20, l21, l22, l23, l24, l25, l26, l27

In [7]:
class HumanProteinAtlasTest:
    # folds with have multile values for train, one value for valid
    def __init__(self, img_height, img_width, mean, std):
        self.files = os.listdir("/home/ubuntu/datasets/human-protein-atlas/test_512by512/")

        self.aug = albumentations.Compose([
            # Resize based on input
            #albumentations.Resize(img_height, img_width, always_apply=True),
            # Normalize
            albumentations.Normalize(mean, std, p=1.0)
        ])


    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, item):
        #image = joblib.load(f"/home/ubuntu/datasets/human-protein-atlas/train_512by512/{self.image_ids[item]}.png")
        image = Image.open(f"/home/ubuntu/datasets/human-protein-atlas/test_512by512/{self.files[item]}")
        # Let's apply the transforms defined in init
        image = self.aug(image=np.array(image))["image"]
        # Transpose the image to fit torch vision channels, so exchange channel locations
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return{
            'image': torch.tensor(image, dtype=torch.float),
            'image_id': self.files[item].replace(".png","")
        }

In [8]:
dataset = HumanProteinAtlasTest(img_height=512, img_width=512,
                             mean=(0.486, 0.456, 0.406),
                             std=(0.229, 0.224, 0.225))

In [9]:
dataset.__getitem__(0)

{'image': tensor([[[-2.1223, -2.1223, -2.1051,  ..., -2.1223, -2.1223, -2.1223],
          [-2.1223, -2.1223, -2.0709,  ..., -2.1223, -2.1223, -2.1223],
          [-2.1223, -2.1223, -2.0024,  ..., -2.1223, -2.1223, -2.1223],
          ...,
          [-2.1223, -2.1223, -2.1223,  ..., -2.0880, -2.1223, -2.1223],
          [-2.1223, -2.1223, -2.1223,  ..., -2.1223, -2.0880, -2.0880],
          [-2.1223, -2.1223, -2.1223,  ..., -2.1051, -2.0709, -2.0709]],
 
         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -1.8782,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -1.7906, -1.7731, -1.6155],
          [-2.0357, -2.0357, -2.0357,  ..., -1.8606, -1.7731, -1.7381],
          [-2.0357, -2.0357, -2.0357,  ..., -1.7731, -1.7731, -1.5455]],
 
         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044,

In [39]:
#model = ResNet34(pretrained=False)
model_0 = model_1 = model_2 = model_3 = model_4 = ResNet50(pretrained=False)
model_name = 'resnet50'

In [16]:
model_0.load_state_dict(torch.load(f"../models/{model_name}_fold0.bin"))
model_0.cuda()
model_0.eval()

ResNet50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d

In [17]:
model_1.load_state_dict(torch.load(f"../models/{model_name}_fold1.bin"))
model_1.cuda()
model_1.eval()

ResNet50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d

In [40]:
model_2.load_state_dict(torch.load(f"../models/{model_name}_fold2.bin"))
model_2.cuda()
model_2.eval()

ResNet50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d

In [41]:
model_3.load_state_dict(torch.load(f"../models/{model_name}_fold3.bin"))
model_3.cuda()
model_3.eval()

ResNet50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d

In [42]:
model_4.load_state_dict(torch.load(f"../models/{model_name}_fold4.bin"))
model_4.cuda()
model_4.eval()

ResNet50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d

In [19]:
data_loader = torch.utils.data.DataLoader(dataset,
                                         batch_size=TEST_BATCH_SIZE,
                                         shuffle=False,
                                         num_workers=4)

*****__Quick experiment starts__*****

In [30]:
tp = torch.rand(6,2)
tp

tensor([[0.1127, 0.1032],
        [0.6925, 0.4034],
        [0.9392, 0.1678],
        [0.8335, 0.9643],
        [0.9051, 0.2442],
        [0.9191, 0.2429]])

In [33]:
# F.softmax??

In [31]:
F.softmax(tp, dim=0) 

tensor([[0.0864, 0.1237],
        [0.1543, 0.1670],
        [0.1974, 0.1320],
        [0.1776, 0.2927],
        [0.1908, 0.1424],
        [0.1935, 0.1423]])

In [44]:
# apply softmax for each row
tf = F.softmax(tp, dim=1)
tf

tensor([[0.5024, 0.4976],
        [0.5718, 0.4282],
        [0.6838, 0.3162],
        [0.4673, 0.5327],
        [0.6595, 0.3405],
        [0.6629, 0.3371]])

In [49]:
# for every row select index of column with max value
ta = torch.argmax(tf,dim=1)
ta

tensor([0, 0, 0, 1, 0, 0])

***** __Quick experiment ends__ *****

__Let's use all folds to predict the output for each test image__

In [51]:
predictions = []

# add tqdm next time
for bi, d in enumerate(data_loader):
    image = d["image"]
    img_id = d["image_id"]
    image = image.to(DEVICE, dtype=torch.float)
    with torch.no_grad():
        # fold 0
        t0, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20, t21, t22, t23, t24, t25, t26, t27 = model_0(image)
        # This stacks array by adding 27 rows below the first one
        tt_0 = torch.cat((t0, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20, t21, t22, t23, t24, t25, t26, t27), 0)
        
        ## fold 1
        t0, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20, t21, t22, t23, t24, t25, t26, t27 = model_1(image)
        tt_1 = torch.cat((t0, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20, t21, t22, t23, t24, t25, t26, t27), 0)
        
        ## fold 2
        t0, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20, t21, t22, t23, t24, t25, t26, t27 = model_2(image)
        tt_2 = torch.cat((t0, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20, t21, t22, t23, t24, t25, t26, t27), 0)
        
        ## fold 3
        t0, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20, t21, t22, t23, t24, t25, t26, t27 = model_3(image)
        tt_3 = torch.cat((t0, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20, t21, t22, t23, t24, t25, t26, t27), 0)
        
        ## fold 4
        t0, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20, t21, t22, t23, t24, t25, t26, t27 = model_4(image)
        tt_4 = torch.cat((t0, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14, t15, t16, t17, t18, t19, t20, t21, t22, t23, t24, t25, t26, t27), 0)
         
        # every tt_n above has dimetion of num_heads * num_classes_for_respective_head, so 28*2
        # we take average of the output and then apply softmax
        tt = F.softmax((tt_0 + tt_1 + tt_2 + tt_3 + tt_4)/5, dim=1)
        
        #print(tt)
        
        # This creates 1D tensor of length 28 with 1s and 0s, for the entire batch, in our case just 1
        tt2 = torch.argmax(tt,dim=1)
        # get the tt2 tensor on cpu, convert to ndarray, find indices where values is 1
        pred_classes = np.where(tt2.cpu().numpy() == 1)
        predictions.append((img_id,pred_classes))

In [52]:
len(predictions)

11702

In [53]:
res_list = []

for idx, (img_name, classes) in enumerate(predictions):
    class_nums = pd.Series(classes)[0]
    if len(str(class_nums))==2:
        class_nums = 0
    tt = (img_name[0]+","+str(class_nums).replace("[","").replace("]","")).replace(", ",",").replace('  ',' ')
    print(idx, tt)
    #file_object.write(tt)
    res_list.append(tt.replace('  ',' '))

0 a504d72c-bacb-11e8-b2b8-ac1f6b6435d0,0 21
1 a4287060-bac7-11e8-b2b7-ac1f6b6435d0,0 12
2 2d9efbe8-bad7-11e8-b2b9-ac1f6b6435d0,0 4 21
3 2c8ef12a-bad4-11e8-b2b8-ac1f6b6435d0,0 23
4 73fd8e68-bac5-11e8-b2b7-ac1f6b6435d0,0
5 541ac51e-bac7-11e8-b2b7-ac1f6b6435d0,0
6 a02db460-bad7-11e8-b2b9-ac1f6b6435d0,0 3
7 2e5c5fd8-bacf-11e8-b2b8-ac1f6b6435d0,0
8 c99e8bd6-bac7-11e8-b2b7-ac1f6b6435d0,0 25
9 c1bdae20-bac6-11e8-b2b7-ac1f6b6435d0,0
10 2bea3fba-bac8-11e8-b2b7-ac1f6b6435d0,0
11 c6f9aa3c-bad6-11e8-b2b9-ac1f6b6435d0,0 21
12 b13a174e-bac8-11e8-b2b8-ac1f6b6435d0,0 2
13 6c884286-bacf-11e8-b2b8-ac1f6b6435d0,0 21
14 21a5995a-bad7-11e8-b2b9-ac1f6b6435d0,0 21 25
15 342ecd1e-bad5-11e8-b2b8-ac1f6b6435d0,0 2
16 63d99998-bacd-11e8-b2b8-ac1f6b6435d0,0 21
17 d2a8a95e-bac8-11e8-b2b8-ac1f6b6435d0,0 21
18 3d139f5c-bad2-11e8-b2b8-ac1f6b6435d0,0 5
19 be8432d8-bacb-11e8-b2b8-ac1f6b6435d0,0 7
20 5e4fb436-bad6-11e8-b2b9-ac1f6b6435d0,0 25
21 bbfad730-bac9-11e8-b2b8-ac1f6b6435d0,0 24
22 deadbca2-bad3-11e8-b2b8-ac1f6b64

459 15a4def0-bacc-11e8-b2b8-ac1f6b6435d0,0 21 25
460 dc5fd918-bad7-11e8-b2b9-ac1f6b6435d0,26
461 7ccce802-bad1-11e8-b2b8-ac1f6b6435d0,0 4 25
462 e4fc2698-bace-11e8-b2b8-ac1f6b6435d0,0 7
463 0dfb7238-bacf-11e8-b2b8-ac1f6b6435d0,0 3 25
464 8bfe8454-baca-11e8-b2b8-ac1f6b6435d0,0 19
465 d990253c-bac6-11e8-b2b7-ac1f6b6435d0,0 25
466 b1bc63e2-bad8-11e8-b2b9-ac1f6b6435d0,0 7
467 1c1d76b6-bad1-11e8-b2b8-ac1f6b6435d0,0 6
468 c722507e-bac5-11e8-b2b7-ac1f6b6435d0,0 2
469 2679a946-bad0-11e8-b2b8-ac1f6b6435d0,0 21
470 451ccf80-bad6-11e8-b2b9-ac1f6b6435d0,0 25
471 6f4a18bc-bad6-11e8-b2b9-ac1f6b6435d0,0 12 21
472 49a521fe-bac9-11e8-b2b8-ac1f6b6435d0,0 25
473 cf1680b2-bad8-11e8-b2b9-ac1f6b6435d0,0 25
474 bee23850-bac7-11e8-b2b7-ac1f6b6435d0,0
475 106a4a5e-bac9-11e8-b2b8-ac1f6b6435d0,0 25
476 b28891c2-bad0-11e8-b2b8-ac1f6b6435d0,5
477 23a83782-bad5-11e8-b2b8-ac1f6b6435d0,0 2 5
478 d19b71fe-bac8-11e8-b2b8-ac1f6b6435d0,0 25
479 a561de46-bad9-11e8-b2b9-ac1f6b6435d0,0 12 21 25
480 dcd1703c-bac8-11e8-b2b8-a

935 d4db5e1a-bac8-11e8-b2b8-ac1f6b6435d0,0
936 dd9188e6-bad6-11e8-b2b9-ac1f6b6435d0,0 19
937 f0a6d40a-bacb-11e8-b2b8-ac1f6b6435d0,0 14
938 9f584d72-bac6-11e8-b2b7-ac1f6b6435d0,0 21
939 bee1beac-bac7-11e8-b2b7-ac1f6b6435d0,0 18 19
940 a32f77f0-bad9-11e8-b2b9-ac1f6b6435d0,0 21
941 3a466a8c-bad9-11e8-b2b9-ac1f6b6435d0,0 14
942 dc7ee4e8-bad7-11e8-b2b9-ac1f6b6435d0,0
943 e309ff72-bace-11e8-b2b8-ac1f6b6435d0,9 10 13
944 b2396766-bace-11e8-b2b8-ac1f6b6435d0,0
945 af161cf0-bad8-11e8-b2b9-ac1f6b6435d0,0 25
946 ee238416-bacc-11e8-b2b8-ac1f6b6435d0,0
947 82c58aae-bad9-11e8-b2b9-ac1f6b6435d0,0 11
948 8f851cd4-bad7-11e8-b2b9-ac1f6b6435d0,0 25
949 487523ea-bacb-11e8-b2b8-ac1f6b6435d0,0 25
950 7ae7982a-bac7-11e8-b2b7-ac1f6b6435d0,0
951 57884f24-bada-11e8-b2b9-ac1f6b6435d0,0 18 25
952 09738310-bad2-11e8-b2b8-ac1f6b6435d0,0
953 f0dd4402-bacd-11e8-b2b8-ac1f6b6435d0,0 2
954 ac0315ee-baca-11e8-b2b8-ac1f6b6435d0,0
955 e9504430-baca-11e8-b2b8-ac1f6b6435d0,0 21
956 bc43a69a-bac9-11e8-b2b8-ac1f6b6435d0,0 23
9

1409 5135b312-bad2-11e8-b2b8-ac1f6b6435d0,0 7
1410 e4eb0874-bad6-11e8-b2b9-ac1f6b6435d0,0 21 25
1411 ea334ab8-bacb-11e8-b2b8-ac1f6b6435d0,0 23
1412 c0c75098-bad0-11e8-b2b8-ac1f6b6435d0,0
1413 176ab0dc-bad8-11e8-b2b9-ac1f6b6435d0,0 21
1414 75ff584e-bada-11e8-b2b9-ac1f6b6435d0,0 23 25
1415 ced8e314-bac5-11e8-b2b7-ac1f6b6435d0,0 2 25
1416 f865ab9a-bad9-11e8-b2b9-ac1f6b6435d0,0 3
1417 ce55886a-bad5-11e8-b2b9-ac1f6b6435d0,0 1
1418 0189c954-bad5-11e8-b2b8-ac1f6b6435d0,0 4
1419 b885428c-bad5-11e8-b2b9-ac1f6b6435d0,0 21
1420 254fb754-bad5-11e8-b2b8-ac1f6b6435d0,0
1421 9fdc326a-bad8-11e8-b2b9-ac1f6b6435d0,0 21
1422 798c8682-bace-11e8-b2b8-ac1f6b6435d0,0 3
1423 0031820a-baca-11e8-b2b8-ac1f6b6435d0,0
1424 6ed58b44-bac9-11e8-b2b8-ac1f6b6435d0,0 23 25
1425 867ad4c8-bacb-11e8-b2b8-ac1f6b6435d0,0
1426 d3ddc6f6-bacd-11e8-b2b8-ac1f6b6435d0,0 12
1427 7c6a0764-bac7-11e8-b2b7-ac1f6b6435d0,0 25
1428 2cb9d35a-bad8-11e8-b2b9-ac1f6b6435d0,0
1429 40ce3564-bad0-11e8-b2b8-ac1f6b6435d0,0
1430 215e7f66-bad2-11e8-b

1890 e768d2d6-baca-11e8-b2b8-ac1f6b6435d0,0 11
1891 a68e8b7c-bac8-11e8-b2b8-ac1f6b6435d0,0 2
1892 2a193426-bad6-11e8-b2b9-ac1f6b6435d0,0
1893 a86c48ba-baca-11e8-b2b8-ac1f6b6435d0,0 21 25
1894 aebf8f5e-bad6-11e8-b2b9-ac1f6b6435d0,0 4
1895 f7bf5e48-bac5-11e8-b2b7-ac1f6b6435d0,0 2 21 25
1896 68b911b8-bad8-11e8-b2b9-ac1f6b6435d0,0
1897 1bab8292-bacf-11e8-b2b8-ac1f6b6435d0,0 23
1898 742ce318-bacd-11e8-b2b8-ac1f6b6435d0,0 25
1899 eb3d8d9a-bacd-11e8-b2b8-ac1f6b6435d0,0 7
1900 e3bdcad2-bacb-11e8-b2b8-ac1f6b6435d0,0 1
1901 e5cacdac-bad5-11e8-b2b9-ac1f6b6435d0,0
1902 4cc91d04-bad3-11e8-b2b8-ac1f6b6435d0,0
1903 e5aa31bc-bad7-11e8-b2b9-ac1f6b6435d0,0 11
1904 b1fbcca2-baca-11e8-b2b8-ac1f6b6435d0,0
1905 46feb5c2-bad4-11e8-b2b8-ac1f6b6435d0,0
1906 91a1573e-bac9-11e8-b2b8-ac1f6b6435d0,0 4
1907 cfbc9804-bad2-11e8-b2b8-ac1f6b6435d0,0 11
1908 dfed3c2e-bad2-11e8-b2b8-ac1f6b6435d0,0
1909 e8ddf43a-bac9-11e8-b2b8-ac1f6b6435d0,0 26
1910 5b8958d2-bac8-11e8-b2b8-ac1f6b6435d0,0
1911 a0649ef6-bad9-11e8-b2b9-ac1f6

...

9725 4ddb80c8-baca-11e8-b2b8-ac1f6b6435d0,0 1 2
9726 6b3a26da-bad7-11e8-b2b9-ac1f6b6435d0,0 21 25
9727 ff5ee18c-baca-11e8-b2b8-ac1f6b6435d0,0 25
9728 9f51de12-bace-11e8-b2b8-ac1f6b6435d0,0 25
9729 268710e0-bacb-11e8-b2b8-ac1f6b6435d0,1 2
9730 bc5cc49e-bad9-11e8-b2b9-ac1f6b6435d0,0 25
9731 25ff8df8-bad7-11e8-b2b9-ac1f6b6435d0,0 2 3
9732 17d0d978-bad0-11e8-b2b8-ac1f6b6435d0,0 25
9733 5d5fd2aa-bad0-11e8-b2b8-ac1f6b6435d0,0
9734 7ce8f730-bac8-11e8-b2b8-ac1f6b6435d0,0
9735 b7afe956-bac7-11e8-b2b7-ac1f6b6435d0,0 14 25
9736 fb71a95a-bad0-11e8-b2b8-ac1f6b6435d0,0
9737 b0953bfa-bad9-11e8-b2b9-ac1f6b6435d0,0 25
9738 5d151f60-bad2-11e8-b2b8-ac1f6b6435d0,0 24
9739 440f97a2-bad2-11e8-b2b8-ac1f6b6435d0,0 23
9740 5bf80fd6-bac6-11e8-b2b7-ac1f6b6435d0,0 14
9741 a5e7f7ec-baca-11e8-b2b8-ac1f6b6435d0,0
9742 5a1ceab6-bac5-11e8-b2b7-ac1f6b6435d0,0 3
9743 be76caf6-bac8-11e8-b2b8-ac1f6b6435d0,0 18 19
9744 24235d90-bac6-11e8-b2b7-ac1f6b6435d0,0
9745 11824f32-bac8-11e8-b2b7-ac1f6b6435d0,0 25
9746 55e00494-bac8-

10179 8360133e-bac6-11e8-b2b7-ac1f6b6435d0,0
10180 9a90c012-bac6-11e8-b2b7-ac1f6b6435d0,0 21 22
10181 ca56c2f2-bacf-11e8-b2b8-ac1f6b6435d0,0 7
10182 bb1cf716-bacf-11e8-b2b8-ac1f6b6435d0,0 3
10183 88eb1d7e-bad2-11e8-b2b8-ac1f6b6435d0,0 25
10184 c49bbe38-bad6-11e8-b2b9-ac1f6b6435d0,0 23
10185 c833d51e-baca-11e8-b2b8-ac1f6b6435d0,0 25
10186 144afe58-bad7-11e8-b2b9-ac1f6b6435d0,0
10187 37414e00-bad5-11e8-b2b8-ac1f6b6435d0,0 5 24
10188 3e6fea40-bac8-11e8-b2b7-ac1f6b6435d0,0 1
10189 a165562a-bad8-11e8-b2b9-ac1f6b6435d0,0 21
10190 6776baaa-bad6-11e8-b2b9-ac1f6b6435d0,0 25
10191 274340ce-bacd-11e8-b2b8-ac1f6b6435d0,0 25
10192 2929711c-bada-11e8-b2b9-ac1f6b6435d0,0 14
10193 73c8aa40-bac5-11e8-b2b7-ac1f6b6435d0,0 4
10194 8886c414-bad2-11e8-b2b8-ac1f6b6435d0,0
10195 5be5db8a-bac7-11e8-b2b7-ac1f6b6435d0,0
10196 8e6f091c-bac9-11e8-b2b8-ac1f6b6435d0,0 21
10197 402d6fda-bada-11e8-b2b9-ac1f6b6435d0,0 2 13
10198 4b107e2c-bad7-11e8-b2b9-ac1f6b6435d0,0 25
10199 8bae20e0-bac5-11e8-b2b7-ac1f6b6435d0,0 25
1

10642 2127cd60-bacb-11e8-b2b8-ac1f6b6435d0,0 21
10643 1b01a70e-bad9-11e8-b2b9-ac1f6b6435d0,0 2
10644 3e53ad0e-bacc-11e8-b2b8-ac1f6b6435d0,0 25
10645 342c9b34-bada-11e8-b2b9-ac1f6b6435d0,0
10646 ec434c4a-bacb-11e8-b2b8-ac1f6b6435d0,0 2 14 25
10647 dadfd67e-bad2-11e8-b2b8-ac1f6b6435d0,0
10648 b9fe9c74-bacd-11e8-b2b8-ac1f6b6435d0,0 14
10649 20f7e0cc-bacc-11e8-b2b8-ac1f6b6435d0,0 25
10650 ad4ecb6a-bac9-11e8-b2b8-ac1f6b6435d0,0 7
10651 0441c9a4-bad9-11e8-b2b9-ac1f6b6435d0,0 5
10652 e85512da-bad5-11e8-b2b9-ac1f6b6435d0,0 21
10653 6cc04b68-bad4-11e8-b2b8-ac1f6b6435d0,2
10654 2fb86cfc-bacd-11e8-b2b8-ac1f6b6435d0,0 25
10655 0d949248-bad9-11e8-b2b9-ac1f6b6435d0,0 23
10656 3f722964-bada-11e8-b2b9-ac1f6b6435d0,0 2
10657 b10fdbf2-bada-11e8-b2b9-ac1f6b6435d0,0
10658 15071e54-bad1-11e8-b2b8-ac1f6b6435d0,0
10659 3c47925c-bad4-11e8-b2b8-ac1f6b6435d0,0
10660 3dd52334-bacd-11e8-b2b8-ac1f6b6435d0,0 2 25
10661 5ab96c64-bacb-11e8-b2b8-ac1f6b6435d0,0 21
10662 7aedc878-bacf-11e8-b2b8-ac1f6b6435d0,0
10663 7128

11103 f571288a-bac8-11e8-b2b8-ac1f6b6435d0,0
11104 8f257b9c-bacf-11e8-b2b8-ac1f6b6435d0,0 21
11105 418d0a6e-bac8-11e8-b2b7-ac1f6b6435d0,0 25
11106 34eefb88-bad6-11e8-b2b9-ac1f6b6435d0,0 21
11107 1a7d2e1a-bada-11e8-b2b9-ac1f6b6435d0,0 25
11108 fb1839a2-baca-11e8-b2b8-ac1f6b6435d0,0 22
11109 18a69b78-bad3-11e8-b2b8-ac1f6b6435d0,0 3
11110 ff9ccd2a-bad0-11e8-b2b8-ac1f6b6435d0,0 21
11111 1ebb230a-bad1-11e8-b2b8-ac1f6b6435d0,0 11
11112 cf18446c-bac7-11e8-b2b7-ac1f6b6435d0,0 21
11113 a0f9a2fc-bada-11e8-b2b9-ac1f6b6435d0,0
11114 5cc321dc-bad3-11e8-b2b8-ac1f6b6435d0,0
11115 17c9f1a2-bad6-11e8-b2b9-ac1f6b6435d0,0 25
11116 17b20f46-bac8-11e8-b2b7-ac1f6b6435d0,0
11117 4bbda434-bad3-11e8-b2b8-ac1f6b6435d0,0 23
11118 52056b7a-bad7-11e8-b2b9-ac1f6b6435d0,5
11119 4dbff7b2-bacb-11e8-b2b8-ac1f6b6435d0,0 11
11120 7ac7d230-bacf-11e8-b2b8-ac1f6b6435d0,0 2
11121 3e7425aa-bace-11e8-b2b8-ac1f6b6435d0,0 25
11122 ce21544c-baca-11e8-b2b8-ac1f6b6435d0,0 23
11123 d8fe9c00-bac8-11e8-b2b8-ac1f6b6435d0,0 14 17
11124 

11560 81757bae-bada-11e8-b2b9-ac1f6b6435d0,0 4
11561 581b96dc-bad3-11e8-b2b8-ac1f6b6435d0,0 3
11562 8a53cb2e-bad3-11e8-b2b8-ac1f6b6435d0,0 7
11563 6234fad6-bacf-11e8-b2b8-ac1f6b6435d0,0 2
11564 ace6a196-bacb-11e8-b2b8-ac1f6b6435d0,0 25
11565 9001de70-bac5-11e8-b2b7-ac1f6b6435d0,0 11
11566 97b2080e-bad2-11e8-b2b8-ac1f6b6435d0,0 1
11567 f1c547be-bac5-11e8-b2b7-ac1f6b6435d0,0
11568 77ab062a-bada-11e8-b2b9-ac1f6b6435d0,0
11569 7bd629c6-bac8-11e8-b2b8-ac1f6b6435d0,0
11570 6d6d0da4-bad8-11e8-b2b9-ac1f6b6435d0,0 25
11571 0a14d402-bad9-11e8-b2b9-ac1f6b6435d0,0 2
11572 71584a26-bad0-11e8-b2b8-ac1f6b6435d0,0 7
11573 b2b49920-bac6-11e8-b2b7-ac1f6b6435d0,0 5
11574 05a677d8-bad7-11e8-b2b9-ac1f6b6435d0,0 19
11575 630a88c8-bace-11e8-b2b8-ac1f6b6435d0,0 25
11576 7c1b4716-bad9-11e8-b2b9-ac1f6b6435d0,0
11577 3a0a69ba-bad4-11e8-b2b8-ac1f6b6435d0,0 23
11578 e17cc0ac-bad0-11e8-b2b8-ac1f6b6435d0,0 23
11579 599ee704-bac7-11e8-b2b7-ac1f6b6435d0,0
11580 a8d73536-bad8-11e8-b2b9-ac1f6b6435d0,0 23
11581 7bba5826-

In [54]:
#file_object.close()
df = pd.DataFrame(res_list)
df

,0
0,"a504d72c-bacb-11e8-b2b8-ac1f6b6435d0,0 21"
1,"a4287060-bac7-11e8-b2b7-ac1f6b6435d0,0 12"
2,"2d9efbe8-bad7-11e8-b2b9-ac1f6b6435d0,0 4 21"
3,"2c8ef12a-bad4-11e8-b2b8-ac1f6b6435d0,0 23"
4,"73fd8e68-bac5-11e8-b2b7-ac1f6b6435d0,0"
...,...
11697,"27aab2d0-bac9-11e8-b2b8-ac1f6b6435d0,0"
11698,"0222964e-bad9-11e8-b2b9-ac1f6b6435d0,0 4"
11699,"98d28d42-bacf-11e8-b2b8-ac1f6b6435d0,0 18 19"
11700,"9857c6ae-bad7-11e8-b2b9-ac1f6b6435d0,0"


In [55]:
df_res = df[0].str.split(",", n = 1, expand = True) 
df_res.shape, df_res

((11702, 2),
                                           0        1
 0      a504d72c-bacb-11e8-b2b8-ac1f6b6435d0     0 21
 1      a4287060-bac7-11e8-b2b7-ac1f6b6435d0     0 12
 2      2d9efbe8-bad7-11e8-b2b9-ac1f6b6435d0   0 4 21
 3      2c8ef12a-bad4-11e8-b2b8-ac1f6b6435d0     0 23
 4      73fd8e68-bac5-11e8-b2b7-ac1f6b6435d0        0
 ...                                     ...      ...
 11697  27aab2d0-bac9-11e8-b2b8-ac1f6b6435d0        0
 11698  0222964e-bad9-11e8-b2b9-ac1f6b6435d0      0 4
 11699  98d28d42-bacf-11e8-b2b8-ac1f6b6435d0  0 18 19
 11700  9857c6ae-bad7-11e8-b2b9-ac1f6b6435d0        0
 11701  34d90798-bada-11e8-b2b9-ac1f6b6435d0        0
 
 [11702 rows x 2 columns])

In [56]:
df_res.columns = ["Id","Predicted"]
df_res

,Id,Predicted
0,a504d72c-bacb-11e8-b2b8-ac1f6b6435d0,0 21
1,a4287060-bac7-11e8-b2b7-ac1f6b6435d0,0 12
2,2d9efbe8-bad7-11e8-b2b9-ac1f6b6435d0,0 4 21
3,2c8ef12a-bad4-11e8-b2b8-ac1f6b6435d0,0 23
4,73fd8e68-bac5-11e8-b2b7-ac1f6b6435d0,0
...,...,...
11697,27aab2d0-bac9-11e8-b2b8-ac1f6b6435d0,0
11698,0222964e-bad9-11e8-b2b9-ac1f6b6435d0,0 4
11699,98d28d42-bacf-11e8-b2b8-ac1f6b6435d0,0 18 19
11700,9857c6ae-bad7-11e8-b2b9-ac1f6b6435d0,0


In [57]:
samp = pd.read_csv("/home/ubuntu/datasets/human-protein-atlas/sample_submission.csv")

In [58]:
samp.head()

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0


In [59]:
actual_submission = samp.merge(df_res, left_on='Id', right_on='Id')

In [60]:
actual_submission.drop(columns='Predicted_x',inplace=True)

In [61]:
actual_submission.head()

,Id,Predicted_y
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0 2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0 5
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0 25


In [62]:
actual_submission.columns = ["Id","Predicted"]
actual_submission.head(10)

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0 2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0 5
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0 25
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,0 4
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,0 4 25
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0 25
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,0 25


In [63]:
actual_submission.to_csv("../results/results_resnet50_v0_cv5.csv",index=False)

In [ ]:
!head ../results/results_resnet50_v0_cv5.csv

Id,Predicted
00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0 2
0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0 5
0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 25
0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25
000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0 25
00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,0 4
001765de-bacd-11e8-b2b8-ac1f6b6435d0,0 4 25
0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0 25
00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0
